In [1]:
# Install required packages and set up Chrome WebDriver
!pip install selenium beautifulsoup4
!apt-get update
!apt-get -q -y install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,247 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-upda

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def scrape_reviews(url, driver):
    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        return None

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    return review_elements

In [5]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # To run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1
stop = False  # Initialize the 'stop' variable

while not stop:
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=1996256&i={page * 20}"

    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        stop = True  # Set 'stop' to True to exit the loop
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    if len(review_elements) == 0:
        stop = True
        break

    for review in review_elements:
        name_element = review.find("p", class_="author js-author")
        comment_element = review.find("div", class_="phml msg-body")
        date_element = review.find("span", class_="timestamp")

        name = name_element.text.strip() if name_element else ""
        comment = comment_element.text.strip() if comment_element else ""
        date = date_element.text.strip() if date_element else ""

        names.append(name)
        comments.append(comment)
        dates.append(date)

    page += 1

In [6]:
# Close the webdriver
driver.quit()

In [7]:
# Create a pandas DataFrame from the scraped data
mini4 = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

In [8]:
# Display the review data
print(mini4)

                Name                                            Comment  \
0         cerb4.5lee  Mouse Rat said:\n\nAndy83n said:\n How many Ro...   
1                 Om  'An all-electric drive, the Charismatic Simpli...   
2              D4rez  Jader1973 said:\n\nPflanzgarten said:\n\nJader...   
3           BFleming  My first impression was "Citroen Cactus done p...   
4   Fetchez la vache  sidesauce said:\n I actually don't mind it!Me ...   
5        DonkeyApple  Jader1973 said:\n\nPflanzgarten said:\n\nJader...   
6            wpa1975    It is ok, I have seen worse especially from BMW   
7            nismo48  I like it, it's funky and chic...a bit like th...   
8           muchacho  Triumph Man said:\n I get the concept of a sma...   
9         RenesisEvo  J4CKO said:\n\nHellbound said:\n\nMaxym said:\...   
10         DevonPaul  Looks lovely.Sadly my interest vanished when I...   
11          benjidog  [quote=Triumph Man]It looks incredibly wk - an...   
12       DonkeyApple  ben

In [9]:
# Save the DataFrame to a CSV file
csv_filename = "mini4.csv"  # You can choose the desired file name
mini4.to_csv(csv_filename, index=False)

In [10]:
# Display a message after saving
print(f"Scraped data saved to {csv_filename}")

Scraped data saved to mini4.csv


In [11]:
import os

In [12]:
# Define the folder name
folder_name = "Colab_Data"

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Create the folder in Google Drive
drive_path = "/content/drive/MyDrive/"
folder_path = os.path.join(drive_path, folder_name)

In [15]:
# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

print(f"Folder '{folder_name}' created in Google Drive")

Folder 'Colab_Data' created in Google Drive


In [16]:
csv_filename = os.path.join(folder_path, "mini4.csv")
mini4.to_csv(csv_filename, index=False)

print(f"CSV file saved to '{csv_filename}'")

CSV file saved to '/content/drive/MyDrive/Colab_Data/mini4.csv'
